## Policy Playground  
This is code for loading and testing saved policies.  
The models '2H300_1' and '2H300_2' were the best policy iterations after a few thousand
iterations of training in their respective runs.  
They did not perform as well as scripted policies (see /P2_Environment/RNN_DDPG/test_rnn_env.ipynb).  
All runs were done with 2 Herds and a total population of 300.

Model 2H300_DDPG was found after 100 Steps by DDPG. Return is oddly low at 7600 for 100 cullsteps on average. Runs on Env.

In [1]:
import tensorflow as tf
import numpy as np

import tensorflow_probability as tfp
from tf_agents.environments import tf_py_environment
from tf_agents.policies import scripted_py_policy

import matplotlib
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
#plt.ioff() 

#from RNN_Env_P2 import Env_P2_N
import sys
sys.path.insert(1, '/home/jovyan/Masterarbeit/reinforce-one/Environment')
from Env import Env

In [2]:
max_episode_length=1000
num_herds = 2   # plotting currently only works for 2 herds
total_population = 300

#saved_pol_1 = tf.compat.v2.saved_model.load('2H300_1')
#saved_pol_2 = tf.compat.v2.saved_model.load('2H300_2')
saved_pol_3 = tf.compat.v2.saved_model.load('2H300_DDPG')#
saved_pol_4 = tf.compat.v2.saved_model.load('2H300_RNN_DDPG')
#py_env = Env_P2_N(num_herds = num_herds, total_population = total_population)
#eval_tf_env = tf_py_environment.TFPyEnvironment(py_env)
env_2 = py_env = Env(num_herds = num_herds, total_population = total_population, 
                                                  fix_episode_length = True, average_episode_length = 100)
eval_2 = tf_py_environment.TFPyEnvironment(env_2)

In [3]:
def test_policy(environment, policy, num_episodes=50):
    if isinstance(environment, tf_py_environment.TFPyEnvironment):
        total_return = 0.0
        cullsteps = 0 
        actions = []
        for e in range(num_episodes):

            time_step = environment.reset()
            if isinstance(policy, scripted_py_policy.ScriptedPyPolicy):
                policy_state = policy.get_initial_state() # remember where in the script we were
            else:
                #print(policy.get_initial_state(batch_size=train_env.batch_size()))
                policy_state = policy.get_initial_state(batch_size=1) # other policies without memory
            episode_return = 0.0
            i=0
            while not time_step.is_last():
                i+=1
                action_step = policy.action(time_step, policy_state)
                for i in range (num_herds, num_herds*2):
                    if action_step.action[0][i] > 0:
                        cullsteps += 1
                        break
                if e == np.int32(num_episodes/2):
                    actions.append(action_step.action[0])
                policy_state = action_step.state
                time_step = environment.step(action_step.action)
                episode_return += time_step.reward

            total_return += episode_return

        avg_return = total_return / num_episodes
        cullsteps /= num_episodes
        return avg_return, cullsteps, actions
    else:
        return None

In [4]:
 def plot_actions(action_list):
        t = np.linspace(0, len(action_list)+10, num=len(action_list))
        fig, (p1,p2) = plt.subplots(1, 2, figsize=(10,7))
        fig.suptitle('Actions over Time')
        p1.set_title('Tests over Time')
        p1.set_xlabel('Time')
        p1.set_ylabel('Number of Tests')
        p1.set_ylim(-0.2, 1.2)
        p2.set_title('Herd Replacements over Time')
        p2.set_xlabel('Time')
        p2.set_ylabel('Replacement Probability')
        p2.set_ylim(-0.2, 1.2) 
        n_tests_h1, n_tests_h2, replace_h1, replace_h2 = [], [], [], []
        for i in range(len(action_list)):
            n_tests_h1.append(action_list[i][0])
            n_tests_h2.append(action_list[i][1])
            replace_h1.append(action_list[i][2])
            replace_h2.append(action_list[i][3])
        p1.plot(t, n_tests_h1, color='grey', label = 'Herd 1', marker = '', linestyle = '-')
        p1.plot(t, n_tests_h2, color='blue', label = 'Herd 2', marker = '', linestyle = '-')
        p2.plot(t, replace_h1, color='grey', label = 'Herd 1', marker = '', linestyle = '-')
        p2.plot(t, replace_h2, color='blue', label = 'Herd 2', marker = '', linestyle = '-')
        p1.legend()
        p2.legend()
        return fig

In [5]:
#ret, cull, act = test_policy(eval_tf_env, saved_pol_3, num_episodes = 200)
ret, cull, act = test_policy(eval_2, saved_pol_3, num_episodes = 200)
fig = plot_actions(act)

Average Return:  tf.Tensor([-7637.5376], shape=(1,), dtype=float32)
Cullsteps:  100.0
First 20 Actions:  [<tf.Tensor: shape=(4,), dtype=float32, numpy=array([0.4608869 , 0.46040496, 0.4605451 , 0.4640402 ], dtype=float32)>, <tf.Tensor: shape=(4,), dtype=float32, numpy=array([0.45918176, 0.4586793 , 0.45886034, 0.4624468 ], dtype=float32)>, <tf.Tensor: shape=(4,), dtype=float32, numpy=array([0.45728552, 0.4567561 , 0.45698187, 0.4606799 ], dtype=float32)>, <tf.Tensor: shape=(4,), dtype=float32, numpy=array([0.45499596, 0.45442995, 0.45470408, 0.45855403], dtype=float32)>, <tf.Tensor: shape=(4,), dtype=float32, numpy=array([0.18975723, 0.18630543, 0.18663567, 0.20819217], dtype=float32)>, <tf.Tensor: shape=(4,), dtype=float32, numpy=array([0.19315258, 0.189715  , 0.19004503, 0.21152884], dtype=float32)>, <tf.Tensor: shape=(4,), dtype=float32, numpy=array([0.18705428, 0.18353686, 0.18381792, 0.20552105], dtype=float32)>, <tf.Tensor: shape=(4,), dtype=float32, numpy=array([0.18664259, 0.18

In [6]:
print('Average Return: ', ret)
print('Cullsteps: ', cull)
print('First 20 Actions: ', act[:19])

In [10]:
ret, cull, act = test_policy(eval_2, saved_pol_4, num_episodes = 200)
fig = plot_actions(act)

In [11]:
print('Average Return: ', ret)
print('Cullsteps: ', cull)
print('First 20 Actions: ', act[:19])

Average Return:  tf.Tensor([-8379.443], shape=(1,), dtype=float32)
Cullsteps:  89.78
First 20 Actions:  [<tf.Tensor: shape=(4,), dtype=float32, numpy=
array([9.999602e-01, 1.000000e+00, 6.020069e-06, 1.758337e-06],
      dtype=float32)>, <tf.Tensor: shape=(4,), dtype=float32, numpy=
array([1.0000000e+00, 1.0000000e+00, 0.0000000e+00, 2.0563602e-06],
      dtype=float32)>, <tf.Tensor: shape=(4,), dtype=float32, numpy=
array([1.000000e+00, 1.000000e+00, 0.000000e+00, 3.993511e-06],
      dtype=float32)>, <tf.Tensor: shape=(4,), dtype=float32, numpy=
array([1.0000000e+00, 1.0000000e+00, 0.0000000e+00, 5.3048134e-06],
      dtype=float32)>, <tf.Tensor: shape=(4,), dtype=float32, numpy=
array([8.2755297e-02, 3.7146211e-03, 9.2038512e-04, 4.7773123e-05],
      dtype=float32)>, <tf.Tensor: shape=(4,), dtype=float32, numpy=array([0.27004048, 0.13506147, 0.00187033, 0.00074077], dtype=float32)>, <tf.Tensor: shape=(4,), dtype=float32, numpy=array([0.36163032, 0.2512799 , 0.00136945, 0.00059852],